# Libraries

In [331]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences

import numpy as np
import os

# Preprocessing

In [332]:
# base path
directory_path = '/mnt/d/GitHub/SSLrecognition/train_data/videos'
# current directory
c_dir = os.getcwd()

# all actions
# actions = np.array(sorted([folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))])) # sorted to follow folder arrangement

# specific actions
# actions = np.array(['alligator', 'flower', 'kiss', 'listen', 'orange'])
actions = np.array(['afternoon', 'house', 'again', 'open', 'kiss', 'sorry'])

In [333]:
# create a dictionary for int representation of actions
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'afternoon': 0, 'house': 1, 'again': 2, 'open': 3, 'kiss': 4, 'sorry': 5}

Note that at this point, we will not access the video folder, only the numpy folder.

In [334]:
sequences, labels = [], []  # sequence -> video, labels -> action
for action in actions:
    no_actions = len(os.listdir(os.path.join(c_dir, 'labels', action)))
    print('Opening path:', os.path.join(c_dir, 'labels', action))
    print(f'Number of instances: {no_actions}')
    for num in range(1, no_actions + 1):
        window = []         # window -> single frame
        file = str(action) + "_" + str(num)
        no_frames_per_action = len(os.listdir(os.path.join(c_dir, 'labels', action, file)))
        print(f'Number of frames in {file}: {no_frames_per_action}')
        for frame_num in range(1, no_frames_per_action + 1):
            res = np.load(os.path.join(c_dir, 'labels', action, file,  "{}.npy".format(frame_num)))     # res -> coordinate key points
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
    print('-'*75)

Opening path: d:\GitHub\SSLrecognition\train_data\labels\afternoon
Number of instances: 40
Number of frames in afternoon_1: 31
Number of frames in afternoon_2: 30
Number of frames in afternoon_3: 30
Number of frames in afternoon_4: 30
Number of frames in afternoon_5: 31
Number of frames in afternoon_6: 31
Number of frames in afternoon_7: 31
Number of frames in afternoon_8: 30
Number of frames in afternoon_9: 31
Number of frames in afternoon_10: 31
Number of frames in afternoon_11: 31
Number of frames in afternoon_12: 31
Number of frames in afternoon_13: 31
Number of frames in afternoon_14: 31
Number of frames in afternoon_15: 31
Number of frames in afternoon_16: 31
Number of frames in afternoon_17: 31
Number of frames in afternoon_18: 31
Number of frames in afternoon_19: 31
Number of frames in afternoon_20: 31
Number of frames in afternoon_21: 31
Number of frames in afternoon_22: 31
Number of frames in afternoon_23: 31
Number of frames in afternoon_24: 31
Number of frames in afternoon_

In [335]:
# due to difference in number of frames, pad x and y
x = np.array(pad_sequences(sequences, dtype = 'float', padding = 'post', value = 0))
y = pad_sequences(to_categorical(labels).astype(int), dtype = 'int', padding = 'post', value = -1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, stratify = y)

In [336]:
input_shape = (x_train.shape[1], x_train.shape[2])

In [337]:
y_test

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1]])

# Models

## LSTM

In [338]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import TensorBoard, TerminateOnNaN, EarlyStopping

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [339]:
# for logging of data with TensorBoard
# log_dir = os.path.join(c_dir, 'Logs')
# tb_callback = TensorBoard(log_dir = log_dir)

# to end training when failure happens ie. loss == nan
term = TerminateOnNaN()

# to stop training early if there is no change in loss
early = EarlyStopping(monitor = 'loss', patience = 5)

In [340]:
def choose_lstm(n):
    if n == 1:
        model = Sequential()
        model.add(LSTM(256, return_sequences = True, input_shape = input_shape))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(512, return_sequences = True)))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dropout(0.2))
        # model.add(LSTM(512, return_sequences = False))
        # model.add(Dropout(0.2))
        # model.add(Dense(512, activation='relu'))
        # model.add(Dense(256, activation='relu'))
        # model.add(Dense(128, activation='relu'))
        # model.add(Dense(64, activation='relu'))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model

    elif n == 2:
        model = Sequential()
        model.add(LSTM(64, return_sequences = True, input_shape = (117, 225)))
        model.add(LSTM(128, return_sequences = True))
        model.add(LSTM(64, return_sequences = False))
        model.add(Dense(64))
        model.add(Dense(32))
        model.add(Dense(8))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model

    elif n == 3:
        model = Sequential()
        model.add(LSTM(128, return_sequences = True, input_shape = (117, 225)))
        model.add(Dropout(0.1))
        model.add(LSTM(64, return_sequences = False))
        model.add(Dropout(0.1))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model
    
    elif n == 4:
        model = Sequential()
        model.add(LSTM(128, return_sequences=True, activation='relu', input_shape = input_shape))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=True, activation='relu'))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=False, activation='relu'))
        # model.add(BatchNormalization())
        model.add(Dense(256, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(actions.shape[0], activation='softmax'))

        return model

In [341]:
model = choose_lstm(1)
opt = keras.optimizers.Adam(learning_rate = 0.001)
model.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ['categorical_accuracy'])
model.summary()

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_60 (LSTM)              (None, 144, 256)          493568    
                                                                 
 dropout_43 (Dropout)        (None, 144, 256)          0         
                                                                 
 bidirectional_8 (Bidirecti  (None, 144, 1024)         3149824   
 onal)                                                           
                                                                 
 dropout_44 (Dropout)        (None, 144, 1024)         0         
                                                                 
 bidirectional_9 (Bidirecti  (None, 1024)              6295552   
 onal)                                                           
                                                                 
 dropout_45 (Dropout)        (None, 1024)            

In [342]:
model.fit(x_train, y_train, epochs = 100, batch_size = 4, callbacks = [term, early])

Epoch 1/100
28/28 [==============================] - 81s 3s/step - loss: 1.1825 - categorical_accuracy: 0.5636
Epoch 2/100
28/28 [==============================] - 77s 3s/step - loss: 0.6162 - categorical_accuracy: 0.7909
Epoch 3/100
28/28 [==============================] - 78s 3s/step - loss: 0.6967 - categorical_accuracy: 0.8091
Epoch 4/100
28/28 [==============================] - 78s 3s/step - loss: 0.6392 - categorical_accuracy: 0.7364
Epoch 5/100
28/28 [==============================] - 78s 3s/step - loss: 0.5298 - categorical_accuracy: 0.8455
Epoch 6/100
28/28 [==============================] - 78s 3s/step - loss: 0.7009 - categorical_accuracy: 0.7636
Epoch 7/100
28/28 [==============================] - 78s 3s/step - loss: 0.5736 - categorical_accuracy: 0.8091
Epoch 8/100
28/28 [==============================] - 77s 3s/step - loss: 0.2541 - categorical_accuracy: 0.8909
Epoch 9/100
28/28 [==============================] - 79s 3s/step - loss: 0.3491 - categorical_accuracy: 0.8636
E

In [343]:
# take model predictions
res = model.predict(x_test)

1/1 [==============================] - 1s 1s/step


In [344]:
res

array([[6.60599828e-01, 1.21929878e-02, 2.41499841e-02, 4.44575772e-02,
        3.30894045e-03, 2.55290598e-01],
       [4.91522908e-01, 1.67969652e-02, 3.31978947e-02, 6.17153011e-02,
        2.33108178e-01, 1.63658768e-01],
       [7.50277102e-01, 2.00800262e-02, 3.83127294e-02, 8.39459300e-02,
        5.08875400e-03, 1.02295458e-01],
       [7.46304810e-01, 2.05135308e-02, 3.71590406e-02, 8.04858580e-02,
        4.41688811e-03, 1.11119851e-01],
       [5.00993757e-03, 9.26144829e-04, 6.75680407e-04, 1.30357919e-03,
        9.90443408e-01, 1.64129760e-03],
       [7.51555026e-01, 1.96979158e-02, 3.70875821e-02, 8.11098143e-02,
        4.85402113e-03, 1.05695657e-01],
       [2.62110263e-01, 1.24672912e-02, 2.26028152e-02, 3.91980261e-02,
        5.68967342e-01, 9.46542323e-02],
       [7.35668778e-01, 2.11492106e-02, 3.82543318e-02, 7.97483996e-02,
        3.90369934e-03, 1.21275514e-01],
       [5.45403047e-04, 2.02737298e-04, 1.27215913e-04, 2.46489362e-04,
        9.98805404e-01, 

In [345]:
y_true = np.argmax(y_test, axis = 1).tolist()
y_pred = np.argmax(res, axis = 1).tolist()

In [346]:
print(y_true)
print(y_pred)

[0, 4, 2, 0, 4, 1, 4, 0, 4, 0, 5, 4, 5]
[0, 0, 0, 0, 4, 0, 4, 0, 4, 0, 0, 4, 0]


In [347]:
print(f'Accuracy: {round(accuracy_score(y_true, y_pred)*100, 1)}%')

Accuracy: 61.5%


In [348]:
# import os
# import shutil

# folder_path = './averaged_np_labels/'
# parent_files = os.listdir(os.path.join(folder_path))
# write_path = './labels/'

# for parent_file in parent_files:
#     parent_path = os.path.join(folder_path, parent_file)
#     export_path = os.path.join(write_path, parent_file)
#     os.mkdir(export_path)
#     print(f'Created new directory: {export_path}')
#     for i in range(1, len(os.listdir(os.path.join(folder_path, parent_file)))+1):
#         new_subfolder = f'{parent_file}_{i}'
#         os.mkdir(os.path.join(export_path, new_subfolder))
#         print(f'Created new subdirectory: {new_subfolder}')

#         source = os.path.join(parent_path, f'video{i}')
#         destination = os.path.join(os.path.join(export_path, new_subfolder))

#         sourcefolder = os.listdir(os.path.join(parent_path, f'video{i}'))
#         for file in sourcefolder:
#             file_to_copy = os.path.join(os.path.join(parent_path, f'video{i}'), file)
#             shutil.copy(file_to_copy, destination)